In [1]:
#Setup for colab to have access to the files, etc.
from google.colab import drive
drive.mount('/content/drive')

# Insert the directory
import sys
sys.path.insert(0,'/content/drive/My Drive/')

import os
os.chdir("/content/drive/My Drive/")

Mounted at /content/drive


In [2]:
!pip install torch
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.9 MB/s eta 0:00:00


In [3]:
import pandas as pd

data_frame = pd.read_csv('labeled_sequences.csv')


In [4]:
len(data_frame)

74632

In [5]:
data_frame.shape

(74632, 8)

In [6]:
data_frame.head()

,db,unique_id,entry_name,protein_name,organism_name,organism_id,sequence,label
0,sp,A6UW25,TRPB_META3,Tryptophan synthase beta chain,Methanococcus aeolicus (strain ATCC BAA-1280 /...,419665,MEYKFGEYGGQYVPEVLMPSLKELEKAYKKYKDDPEFKEELEYYLK...,Archaea
1,sp,Q57998,Y578_METJA,Uncharacterized protein MJ0578,Methanocaldococcus jannaschii (strain ATCC 430...,243232,MDYLGIIMKIAIISGKGGVGKSSISTSLAKLFSKEFNIVALDCDVD...,Archaea
2,sp,Q59033,Y1639_METJA,Uncharacterized protein MJ1639,Methanocaldococcus jannaschii (strain ATCC 430...,243232,MYFVIKGDSIMYFVLRLGGDLSLYIPAKYRPFFEPLDKNSPYNIKV...,Archaea
3,sp,Q5JGR4,RS28_THEKO,30S ribosomal protein S28e,Thermococcus kodakarensis (strain ATCC BAA-918...,69014,MSDEGYPAEVIEIVGRTGVTGGVTQVKVRILEGRDKGRVIRRNVKG...,Archaea
4,sp,Q5JI38,SPEBH_THEKO,N(1)-aminopropylagmatine ureohydrolase,Thermococcus kodakarensis (strain ATCC BAA-918...,69014,MEFLYTYETLKLEFPLVEPEKARFILLGVPFDGTTSYKAGARFGPT...,Archaea


In [7]:
data = [(row['sequence'], row['label']) for _, row in data_frame.iterrows()]

In [8]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_data, valid_data = train_test_split(train_data, test_size=0.2, random_state=42)

In [9]:
num_labels = len(set(data_frame['label']))
print(f'Number of labels: {num_labels}')

Number of labels: 8


In [10]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1


In [11]:
!pip install git+https://github.com/facebookresearch/esm.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/esm.git to /tmp/pip-req-build-g_znnc18
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/esm.git /tmp/pip-req-build-g_znnc18
  Resolved https://github.com/facebookresearch/esm.git to commit a944dc53a9774e5ebfab4be1a395c2c516325e37
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fair-esm: filename=fair_esm-2.0.1-py3-none-any.whl size=104639 sha256=5c891e7face056f18ae927d2919840c7b21188163dd3e5b0cd451b54f326b6bf
  Stored in directory: /tmp/pip-ephem-wheel-cache-bs5zsg69/wheels/6e/14/f7/66c6a330030b05645f124b280232f6d20445a2956af58efd39
Successfully built fair-esm


In [12]:
import esm

In [13]:
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()


Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm1b_t33_650M_UR50S.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm1b_t33_650M_UR50S-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm1b_t33_650M_UR50S-contact-regression.pt


In [14]:
# Freeze the weights of the initial layers
for param in model.parameters():
    param.requires_grad = False


In [15]:
# Replace the final layers with new layers for our task
import torch
num_classes = 8
model.classification_head = torch.nn.Linear(1280, num_classes)

In [24]:
# Load and preprocess the data
df = pd.read_csv('labeled_sequences.csv')
sequences = df['sequence'].tolist()
labels = df['label'].tolist()

inputs = []
for sequence in sequences:
    encoded_sequence = alphabet.encode(sequence)
    inputs.append(encoded_sequence)

In [25]:
# Split the data into training, validation, and test sets
# 74632
train_inputs, val_inputs, test_inputs = inputs[:50000], inputs[50000:64000], inputs[64000:]
train_labels, val_labels, test_labels = labels[:50000], labels[50000:64000], labels[64000:]
